# Artificial Neural Network

We remind that if the user is using Google Colaboratory, he must change the execution type to GPU, see the README.

In [ ]:
#Libraries for data processing
import numpy as np 
import pandas as pd

#Libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set(color_codes = True)
sns.set(font_scale=1.5) #fixing font size

#Libraries for artificial neural network
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Normalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import Callback

In [ ]:
from logging import RootLogger
# Mount Google Drive
from google.colab import drive #import drive from google colab

root = "/content/drive"     #default location for the drive

drive.mount(root)           #we mount the google drive at /content/drive

import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive
my_google_drive_path = "MyDrive/StudentProject2023"

project_path = join(root, my_google_drive_path)

##### We import our training data (all nuclei except Z=10,38,54,68,82) and create rescaled inputs based on the dataframe columns for the ANN

In [ ]:
train_data = pd.read_csv(join(project_path,"rescaled_data/train_rescaled_data.csv"), sep=";")

In [ ]:
#First inputs
target = train_data["rescaled_ame_BE"]
n_input = train_data["rescaled_N"]
z_input = train_data["rescaled_Z"]

#Liquid drop inputs
surf_input = train_data["rescaled_Surf"]
asym_input = train_data["rescaled_Asym"]
coul_input = train_data["rescaled_Coul"]
pair_input = train_data["rescaled_Pair"]

#Other inputs that may help
z_parity_input = train_data["rescaled_Z_parity"]
n_parity_input = train_data["rescaled_N_parity"]
z_distance_input = train_data["rescaled_Z_distance"]  
n_distance_input = train_data["rescaled_N_distance"]
S1p_input = train_data["rescaled_ame_S1p"]
S1n_input = train_data["rescaled_ame_S1n"]
S2p_input = train_data["rescaled_ame_S2p"]
S2n_input = train_data["rescaled_ame_S2n"]

##### We create a function that will create the ANN. It takes 3 parameters: the number of inputs, the number of layers and the number of neurons per layer desired.

In [ ]:
def create_model(num_inputs, num_layers, num_neurons):
    inputs = [keras.layers.Input(shape=(1,)) for i in range(num_inputs)]
    merged = keras.layers.Concatenate()(inputs)

    dense = merged
    for i in range(num_layers):
        dense = Dense(num_neurons, activation="relu")(dense)
    
    output = Dense(1, activation="relu")(dense)
    model = keras.models.Model(inputs, output)
    return model

##### We use this function to create an ANN with 12 inputs, 14 hidden layers with 100 neurons each.

In [ ]:
model4 = create_model(12,14,100) 

In [ ]:
model4.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error")

##### We create a class that will allow us to stop the training of the model when a specific loss is reach. Otherwise, the training will continue until the last epoch.

In [ ]:
class EarlyStoppingByLossValue(Callback):
    def __init__(self, value=0.00000009):
        self.value = value

    def on_epoch_end(self, epoch, logs={}):
        current_loss = logs.get("loss")
        if current_loss < self.value:
            self.model.stop_training = True
            print("Early stopping by loss value at epoch", epoch)

early_stopping = EarlyStoppingByLossValue()

##### We train the model and plot the evolution of the loss among epochs.

In [ ]:
history4=model4.fit(x=([n_input, z_input, surf_input, coul_input, asym_input,
                        pair_input, S1n_input, S1p_input, S2n_input, S2p_input, 
                        z_distance_input, n_distance_input]),
                        y=target, epochs=2000, shuffle=True,
                        verbose=2, callbacks=[early_stopping] )

plt.figure(figsize =(20,13))
plt.yscale('log')
plt.legend('labels')

plt.plot(history4.history["loss"])
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.show()
#Be careful : loss is mean_squared_error, not RMSE

##### We rescale the target data.

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

rescaled_data = pd.read_csv(join(project_path,"rescaled_data/rescaled_data.csv"), sep=";")

rescaled_target = scaler.fit_transform(pd.Series.to_numpy(rescaled_data["ame_BE"]).reshape(-1,1))

#### Here, we predict the binding energy for all the nuclei used for training, and we compute the difference between the prediction and experimental data.

In [ ]:
train_predictions = model4.predict(x=([n_input, z_input, surf_input, coul_input,
                                       asym_input, pair_input, S1n_input, S1p_input,  
                                       S2n_input, S2p_input, z_distance_input,
                                       n_distance_input]), verbose=0)


train_rescaled_predictions = [(i - scaler.min_)/scaler.scale_ for i in train_predictions]


train_data["BE_Predictions"] = np.double(train_rescaled_predictions)
train_data["Difference_BE_AME_ANN"] = train_data["ame_BE"] - train_data["BE_Predictions"]
train_data["Difference_BE_DZ_AME"] = train_data["dz_BE"] - train_data["ame_BE"]

##### We will now do the same operation as above, but we will use the model to predict binding energies for the nuclei it has never trained on (Z=10,38,54,68,82) (validation data)

In [ ]:
validation_data = pd.read_csv(join(project_path,"rescaled_data/validation_rescaled_data.csv"), sep=";")

test_target = validation_data["rescaled_ame_BE"]
test_n_input = validation_data["rescaled_N"]
test_z_input = validation_data["rescaled_Z"]
test_coul_input = validation_data["rescaled_Coul"]
test_surf_input = validation_data["rescaled_Surf"]
test_asym_input = validation_data["rescaled_Asym"]
test_pair_input = validation_data["rescaled_Pair"]
test_z_parity_input = validation_data["rescaled_Z_parity"]
test_n_parity_input = validation_data["rescaled_N_parity"]
test_z_distance_input = validation_data["rescaled_Z_distance"]
test_n_distance_input = validation_data["rescaled_N_distance"]
test_S1p_input = validation_data["rescaled_ame_S1p"]
test_S1n_input = validation_data["rescaled_ame_S1n"]
test_S2p_input = validation_data["rescaled_ame_S2p"]
test_S2n_input = validation_data["rescaled_ame_S2n"]

In [ ]:
validation_predictions = model4.predict(x=([test_n_input, test_z_input,
                                            test_surf_input, test_coul_input,
                                            test_asym_input, test_pair_input,  
                                            test_S1n_input, test_S1p_input,
                                            test_S2n_input, test_S2p_input,
                                            test_z_distance_input,
                                            test_n_distance_input]))

validation_rescaled_predictions = [ (i - scaler.min_)/scaler.scale_ for i in validation_predictions]

validation_data["BE_Predictions"] = np.double(validation_rescaled_predictions)
validation_data["Difference_BE_AME_ANN"] = validation_data["ame_BE"] - validation_data["BE_Predictions"]


##### Here, we compute the values for Sn, Sp, S2n et S2p with the predicted binding energies for both datasets (training and validation)

In [ ]:
train_data=train_data.sort_values(by=['A','N'], ascending=True)
train_data.head(15)

In [ ]:
train_data=train_data.sort_values(by=['N','Z'], ascending=True)
train_data['Prediction_S2p'] = train_data['BE_Predictions'] - train_data['BE_Predictions'].shift(2)

train_data = train_data.sort_values(by=['A','N'], ascending=True)
train_data['Prediction_S2n'] = train_data['BE_Predictions'] - train_data['BE_Predictions'].shift(2)
train_data['Prediction_S1n'] = train_data['BE_Predictions'] - train_data['BE_Predictions'].shift(1)

train_data["Difference_S2n_AME_Predictions"] = train_data["ame_S2n"] - train_data["Prediction_S2n"]
train_data["Difference_S2p_AME_Predictions"] = train_data["ame_S2p"] - train_data["Prediction_S2p"]

train_data["Difference_S2n_DZ_Predictions"] = train_data["dz_S2n"] - train_data["Prediction_S2n"]
train_data["Difference_S2p_DZ_Predictions"] = train_data["dz_S2p"] - train_data["Prediction_S2p"]

In [ ]:
validation_data.sort_values(by=['N','Z'], ascending=True)
validation_data['Prediction_S2p'] = validation_data['BE_Predictions'] - validation_data['BE_Predictions'].shift(2)

validation_data = validation_data.sort_values(by=['A','N'], ascending=True)
validation_data['Prediction_S2n'] = validation_data['BE_Predictions'] - validation_data['BE_Predictions'].shift(2)
validation_data['Prediction_S1n'] = validation_data['BE_Predictions'] - validation_data['BE_Predictions'].shift(1)

validation_data["Difference_S2n_AME_Predictions"] = validation_data["ame_S2n"] - validation_data["Prediction_S2n"]
validation_data["Difference_S2p_AME_Predictions"] = validation_data["ame_S2p"] - validation_data["Prediction_S2p"]

validation_data["Difference_S2n_DZ_Predictions"] = validation_data["dz_S2n"] - validation_data["Prediction_S2n"]
validation_data["Difference_S2p_DZ_Predictions"] = validation_data["dz_S2p"] - validation_data["Prediction_S2p"]

#### We save the predictions into .csv and we will plot them on another notebook

In [ ]:
train_final_csv = train_data.to_csv(join(project_path,"final_data/train_final_data.csv"),sep=";")
validation_final_csv = validation_data.to_csv(join(project_path,"final_data/validation_final_data.csv"),sep=";")